In [5]:
# Import pandas and our pymongo library, which lets us connect our Flask app to our Mongo database.
import pandas as pd
import pymongo
import requests

In [6]:
#Read in the Heart Disease .CSV Dataset change Yes/No in data to boolean True/False 
data = pd.read_csv('./heart_2020_cleaned.csv').replace({'Yes': True, 'No': False})

In [7]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [50]:
#Refresh the collections in Heart_db
#heartdb.mvf.drop()
#heartdb.comorb_rate.drop()
#heartdb.lifestyle_rate.drop()
#heartdb.heartData.drop()

In [8]:
# Export data to MongoDB
heartdb = client.heart_db

In [4]:
collection = heartdb.heartData
data_dict = data.to_dict("records")
collection.insert_many(data_dict)

In [9]:
heartData = list(heartdb.heartData.find())
finalData = []
for data in heartData:
    del data['_id']
    finalData.append(data)

In [10]:
df = pd.DataFrame(finalData)
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,False,16.60,True,False,False,3.0,30.0,False,Female,55-59,White,True,True,Very good,5.0,True,False,True
1,False,20.34,False,False,True,0.0,0.0,False,Female,80 or older,White,False,True,Very good,7.0,False,False,False
2,False,26.58,True,False,False,20.0,30.0,False,Male,65-69,White,True,True,Fair,8.0,True,False,False
3,False,24.21,False,False,False,0.0,0.0,False,Female,75-79,White,False,False,Good,6.0,False,False,True
4,False,23.71,False,False,False,28.0,0.0,True,Female,40-44,White,False,True,Very good,8.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,True,27.41,True,False,False,7.0,0.0,True,Male,60-64,Hispanic,True,False,Fair,6.0,True,False,False
319791,False,29.84,True,False,False,0.0,0.0,False,Male,35-39,Hispanic,False,True,Very good,5.0,True,False,False
319792,False,24.24,False,False,False,0.0,0.0,False,Female,45-49,Hispanic,False,True,Good,6.0,False,False,False
319793,False,32.81,False,False,False,0.0,0.0,False,Female,25-29,Hispanic,False,False,Good,12.0,False,False,False


In [8]:
#Identify the number of males with heart disease and return the percent 
male = df[df['Sex'] == 'Male']
# male.head()
male['HeartDisease'].value_counts()
male_no = male['HeartDisease'].value_counts()[False]
# male_no
male_yes = male['HeartDisease'].value_counts()[True]
# male_yes
male_per = (male_yes /(male_no+male_yes))
male_per

0.10618461740904007

In [ ]:
male['HeartDisease'].value_counts()

In [9]:
#Identify the number of males with heart disease and return the percent 
df = pd.DataFrame(finalData)
female = df[df['Sex'] == 'Female']
# female.head()
female['HeartDisease'].value_counts()
female_no = female['HeartDisease'].value_counts()[False]
# female_no
female_yes = female['HeartDisease'].value_counts()[True]
# female_yes
female_per = (female_yes /(female_no + female_yes))
female_per

0.06694675367241738

In [10]:
#Create dictionary that will be loaded as a collection
mvf = {"male": male_per, "female":female_per}
mvf

{'male': 0.10618461740904007, 'female': 0.06694675367241738}

In [ ]:
df.corr()

In [11]:
# Export data to MongoDB
heartdb = client.heart_db
collection = heartdb.mvf
new_item = collection.insert_one(mvf)


In [12]:
df.columns

Index(['HeartDisease', 'BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
       'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
       'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime',
       'Asthma', 'KidneyDisease', 'SkinCancer'],
      dtype='object')

In [52]:
#Isolate the Comorbidity diseases 
co_morb_cols = ['Stroke','Diabetic', 'Asthma', 'KidneyDisease', 'SkinCancer']

In [53]:
df[co_morb_cols].sum(axis=1)
df["CoMorb"] = df[co_morb_cols].sum(axis=1)

C:\Users\huntl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
C:\Users\huntl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [54]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Lifestyle,CoMorb
0,False,16.60,True,False,False,3.0,30.0,False,Female,55-59,White,True,True,Very good,5.0,True,False,True,2,2
1,False,20.34,False,False,True,0.0,0.0,False,Female,80 or older,White,False,True,Very good,7.0,False,False,False,1,1
2,False,26.58,True,False,False,20.0,30.0,False,Male,65-69,White,True,True,Fair,8.0,True,False,False,2,1
3,False,24.21,False,False,False,0.0,0.0,False,Female,75-79,White,False,False,Good,6.0,False,False,True,0,1
4,False,23.71,False,False,False,28.0,0.0,True,Female,40-44,White,False,True,Very good,8.0,False,False,False,2,0


In [55]:
comorb_count = df.groupby("CoMorb")['HeartDisease'].count()
comorb_count

CoMorb
0    236273
1     71866
2     10371
3      1209
4        76
Name: HeartDisease, dtype: int64

In [56]:
comorb_sum = df.groupby("CoMorb")['HeartDisease'].sum()
comorb_sum

CoMorb
0    13959
1     9749
2     3042
3      568
4       55
Name: HeartDisease, dtype: int64

In [57]:
c_rate = comorb_sum/comorb_count
comorb_rate = c_rate.to_dict()

In [58]:
def map_fields(obj):
    new_obj = {}
    mapping_dict = {0: 'A0', 1: 'A1', 2: 'A2', 3:'A3', 4: 'A4'}
    for key in obj.keys():
        if key in mapping_dict:
            new_key = mapping_dict[key]
        else:
            new_key = key
        new_obj[new_key] = obj[key]
    return new_obj

In [59]:
all_comorb_dict = map_fields(comorb_rate)
all_comorb_dict

{'A0': 0.05907996258565304,
 'A1': 0.13565524726574457,
 'A2': 0.29331790569858257,
 'A3': 0.46980976013234077,
 'A4': 0.7236842105263158}

In [ ]:
df

In [60]:
comorb_G_sum = df.groupby(["CoMorb","Sex"])['HeartDisease'].sum()
comorb_G_sum

CoMorb  Sex   
0       Female    5337
        Male      8622
1       Female    4121
        Male      5628
2       Female    1456
        Male      1586
3       Female     295
        Male       273
4       Female      25
        Male        30
Name: HeartDisease, dtype: int64

In [61]:
comorb_G_count = df.groupby(["CoMorb","Sex"])['HeartDisease'].count()
comorb_G_count

CoMorb  Sex   
0       Female    121080
        Male      115193
1       Female     40066
        Male       31800
2       Female      5917
        Male        4454
3       Female       703
        Male         506
4       Female        39
        Male          37
Name: HeartDisease, dtype: int64

In [62]:
gen_comorb_rate = comorb_G_sum/comorb_G_count
gen_comorb_rate

CoMorb  Sex   
0       Female    0.044078
        Male      0.074848
1       Female    0.102855
        Male      0.176981
2       Female    0.246071
        Male      0.356084
3       Female    0.419630
        Male      0.539526
4       Female    0.641026
        Male      0.810811
Name: HeartDisease, dtype: float64

In [63]:
test2 = gen_comorb_rate.to_dict()


In [38]:
def map_fields2(obj):
    new_obj = {}
    mapping_dict = {(0, 'Female'): 'F0',(0, 'Male'): 'M0', 
                    (1, 'Female'): 'F1',(1, 'Male'): 'M1',
                    (2, 'Female'): 'F2',(2, 'Male'): 'M2', 
                    (3, 'Female'): 'F3',(3, 'Male'): 'M3',
                    (4, 'Female'): 'F4',(4, 'Male'): 'M4', }
    for key in obj.keys():
        if key in mapping_dict:
            new_key = mapping_dict[key]
        else:
            new_key = key
        new_obj[new_key] = obj[key]
    return new_obj

In [64]:
mf_comorb_dict = map_fields2(test2)
mf_comorb_dict

{'F0': 0.044078295341922695,
 'M0': 0.07484829807366768,
 'F1': 0.10285528877352369,
 'M1': 0.1769811320754717,
 'F2': 0.2460706439073855,
 'M2': 0.35608441850022454,
 'F3': 0.41963015647226176,
 'M3': 0.5395256916996047,
 'F4': 0.6410256410256411,
 'M4': 0.8108108108108109}

In [65]:
all_comorb_dict.update(mf_comorb_dict)
all_comorb_dict

{'A0': 0.05907996258565304,
 'A1': 0.13565524726574457,
 'A2': 0.29331790569858257,
 'A3': 0.46980976013234077,
 'A4': 0.7236842105263158,
 'F0': 0.044078295341922695,
 'M0': 0.07484829807366768,
 'F1': 0.10285528877352369,
 'M1': 0.1769811320754717,
 'F2': 0.2460706439073855,
 'M2': 0.35608441850022454,
 'F3': 0.41963015647226176,
 'M3': 0.5395256916996047,
 'F4': 0.6410256410256411,
 'M4': 0.8108108108108109}

In [66]:
# Export data to MongoDB
heartdb = client.heart_db
collection = heartdb.comorb_rate
new_item = collection.insert_one(all_comorb_dict)

In [67]:
df.columns

In [17]:
#Isolate the Lifestyles 
life_style = ['Smoking', 'AlcoholDrinking', 'DiffWalking',
 'PhysicalActivity']

In [18]:
df[life_style].sum(axis=1)
df["Lifestyle"] = df[life_style].sum(axis=1)

In [19]:
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,Lifestyle
0,False,16.60,True,False,False,3.0,30.0,False,Female,55-59,White,True,True,Very good,5.0,True,False,True,2
1,False,20.34,False,False,True,0.0,0.0,False,Female,80 or older,White,False,True,Very good,7.0,False,False,False,1
2,False,26.58,True,False,False,20.0,30.0,False,Male,65-69,White,True,True,Fair,8.0,True,False,False,2
3,False,24.21,False,False,False,0.0,0.0,False,Female,75-79,White,False,False,Good,6.0,False,False,True,0
4,False,23.71,False,False,False,28.0,0.0,True,Female,40-44,White,False,True,Very good,8.0,False,False,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,True,27.41,True,False,False,7.0,0.0,True,Male,60-64,Hispanic,True,False,Fair,6.0,True,False,False,2
319791,False,29.84,True,False,False,0.0,0.0,False,Male,35-39,Hispanic,False,True,Very good,5.0,True,False,False,2
319792,False,24.24,False,False,False,0.0,0.0,False,Female,45-49,Hispanic,False,True,Good,6.0,False,False,False,1
319793,False,32.81,False,False,False,0.0,0.0,False,Female,25-29,Hispanic,False,False,Good,12.0,False,False,False,0


In [23]:
ls_count = df.groupby("Lifestyle")['HeartDisease'].count()
ls_count

Lifestyle
0     25283
1    165887
2    106509
3     21317
4       799
Name: HeartDisease, dtype: int64

In [22]:
ls_sum = df.groupby("Lifestyle")['HeartDisease'].sum()
ls_sum

Lifestyle
0     1606
1    10283
2    12154
3     3216
4      114
Name: HeartDisease, dtype: int64

In [32]:
ls_rate = ls_sum/ls_count
ls_rate = ls_rate.to_dict()

In [27]:
ls_G_sum = df.groupby(["Lifestyle","Sex"])['HeartDisease'].sum()

In [28]:
ls_G_count = df.groupby(["Lifestyle","Sex"])['HeartDisease'].count()

In [36]:
ls_mf = ls_G_sum/ls_G_count
ls_mf = ls_mf.to_dict()

In [45]:
lsr_dict = map_fields(ls_rate)
lsr_dict

{'A0': 0.06352094292607681,
 'A1': 0.06198797976936107,
 'A2': 0.11411242242439606,
 'A3': 0.15086550640334007,
 'A4': 0.14267834793491865}

In [46]:
lsrmf_dict = map_fields2(ls_mf)
lsrmf_dict

{'F0': 0.053662122427158516,
 'M0': 0.07781761798623897,
 'F1': 0.04940976569486675,
 'M1': 0.0767097120278421,
 'F2': 0.08895400053621356,
 'M2': 0.1383102171630657,
 'F3': 0.12252854358117515,
 'M3': 0.17981790591805766,
 'F4': 0.116751269035533,
 'M4': 0.16790123456790124}

In [47]:
lsr_dict.update(lsrmf_dict)
lsr_dict

{'A0': 0.06352094292607681,
 'A1': 0.06198797976936107,
 'A2': 0.11411242242439606,
 'A3': 0.15086550640334007,
 'A4': 0.14267834793491865,
 'F0': 0.053662122427158516,
 'M0': 0.07781761798623897,
 'F1': 0.04940976569486675,
 'M1': 0.0767097120278421,
 'F2': 0.08895400053621356,
 'M2': 0.1383102171630657,
 'F3': 0.12252854358117515,
 'M3': 0.17981790591805766,
 'F4': 0.116751269035533,
 'M4': 0.16790123456790124}

In [49]:
# Export data to MongoDB
heartdb = client.heart_db
collection = heartdb.lifestyle_rate
new_item = collection.insert_one(lsr_dict)